In [ ]:
def compute_2d_pdf_from_data(data: np.ndarray, 
                              x_bounds: tuple, 
                              y_bounds: tuple, 
                              num_bins: int = 100) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Вычисляет сглаженную PDF из двумерного массива точек с сохранением сингулярностей
    через сглаживание CDF и последующее численное дифференцирование.
    """
    # Сетка
    x = np.linspace(*x_bounds, num_bins)
    y = np.linspace(*y_bounds, num_bins)
    dx = x[1] - x[0]
    dy = y[1] - y[0]
    X, Y = np.meshgrid(x, y, indexing='ij')

    # Кумулятивная вероятность: по всем точкам ниже
    cdf_grid = np.zeros_like(X)
    for i in range(num_bins):
        for j in range(num_bins):
            cdf_grid[i, j] = np.mean((data[:, 0] <= x[i]) & (data[:, 1] <= y[j]))

    # Сглаживание с сохранением краёв
    cdf_smooth = denoise_bilateral(cdf_grid, sigma_color=0.05, sigma_spatial=3, multichannel=False)

    # Численная производная: приближённая PDF
    pdf_estimate = np.gradient(np.gradient(cdf_smooth, dx, axis=0), dy, axis=1)

    # Убираем отрицательные шумы
    pdf_clipped = np.clip(pdf_estimate, 0, None)

    # Нормировка
    normalization = np.sum(pdf_clipped) * dx * dy
    pdf_clipped /= normalization

    return X, Y, pdf_clipped


In [ ]:

def get_pdf(data, a, b, num_points1=1000, num_points2=200):
    from scipy.interpolate import Akima1DInterpolator
    from scipy.integrate import quad

    data_sorted = np.sort(data[(data >= a) & (data <= b)])

    xs_cdf = np.linspace(a, b, num_points1)
    cdf = np.searchsorted(data_sorted, xs_cdf, side='right') / len(data_sorted)

    cdf_interp = Akima1DInterpolator(xs_cdf, cdf)

    xs_pdf = np.linspace(a, b, num_points2)
    dcdf = np.gradient(cdf_interp(xs_pdf), xs_pdf)
    pdf_vals = np.clip(dcdf, 0, None)
    dx = np.diff(xs_pdf)
    dx = np.append(dx, dx[-1])
    integral = np.sum(pdf_vals * dx)
    pdf_vals /= integral

    pdf_function_raw = Akima1DInterpolator(xs_pdf, pdf_vals)
    integral_func, _ = quad(pdf_function_raw, a, b, limit=200)
    pdf_function = lambda x: np.clip(pdf_function_raw(x), 0, None) / integral_func

    return xs_pdf, pdf_vals, pdf_function
